In [18]:
from aws_helper_functions import aws_helper_functions
from sklearn.neighbors import BallTree
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import torch
from torch import nn
from pyro.nn import PyroModule
import pyro.distributions as dist
import pyro

assert issubclass(PyroModule[nn.Linear], nn.Linear)
assert issubclass(PyroModule[nn.Linear], PyroModule)

In [2]:
def _retrieve_nearest_census_tract_numbers(df, local_mode=''):
    # retrive census tract information from redshift
    df_census = aws_helper_functions.read_from_redshift('SELECT * FROM raw_data_science.raw_commute_census_tracts_lat_long', local_mode=local_mode)
    # Create a BallTree for the census tract latitudes and longitudes
    tree = BallTree(df_census[['lat_orig', 'long_orig']].values, leaf_size=40)
    
    #drop rows in df where students_home__latitude__s or students_home__longitude__s is null
    #df = df.dropna(subset=['latitude', 'longitude'])
    df['latitude'] = np.where(df['latitude'].isna()==True, 40.776676, df['latitude'])
    df['longitude'] = np.where(df['longitude'].isna()==True, -73.971321, df['longitude'])
    
    distances, indices = tree.query(df[['latitude', 'longitude']].values, k=1)
    df.loc[:, 'boro_int'] = df_census.loc[indices.flatten(), 'boro_int'].values.copy()
    df.loc[:, 'census_tract_int'] = df_census.loc[indices.flatten(), 'census_tract_int'].values.copy()
    return df

def _get_commute_time(df, local_mode=''):
    #retrieve school-census tract commute times from redshift
    df_commutes = aws_helper_functions.read_from_redshift('SELECT * FROM raw_data_science.raw_commute_census_tracts_to_schools', local_mode=local_mode)
    
    schools = _get_schools()
    df_commutes = _replace_with_keys(df_commutes, 'school', schools)
    #set df_commutes.time_walking_min, time_transit_min, and time_driving_min to numeric
    df_commutes['time_walking_min'] = pd.to_numeric(df_commutes['time_walking_min'], errors='coerce')
    df_commutes['time_transit_min'] = pd.to_numeric(df_commutes['time_transit_min'], errors='coerce')
    df_commutes['time_driving_min'] = pd.to_numeric(df_commutes['time_driving_min'], errors='coerce')
    
    df_commutes['commute_time'] = df_commutes[['time_walking_min', 'time_transit_min']].min(axis=1)
    df_commutes = df_commutes[['boro_int', 'census_tract_int', 'school', 'commute_time']]
    
    df = df.merge(df_commutes, how='left', on=['boro_int', 'census_tract_int', 'school'])
    df['commute_time'] = np.where(df['commute_time']>120,30,df['commute_time'])
    return df

def _get_schools():
    schools = {
        'SA Bed-Stuy 2': 'BED-STUY2',
        'SA Bed-Stuy': 'BED-STUY2',
        'SA Bed-Stuy Middle School': 'BED-STUY_MIDDLE_SCHOOL',
        'SA Bensonhurst': 'BENSONHURST',
        'SA Bergen Beach':'BERGEN_BEACH',
        'SA Bronx 1 Middle School': 'BRONX1',
        'SA Bronx 1': 'BRONX1',
        'SA Bronx Middle School': 'BRONX_MIDDLE_SCHOOL',
        'SA Bronx 2': 'BRONX2',
        'SA Bronx 2 Middle School': 'BRONX2_MIDDLE_SCHOOL',
        'SA Bronx 3': 'BRONX3',
        'SA Bronx 4': 'BRONX4',
        'SA Bronx 5': 'BRONX5',
        'SA Bronx 5 Upper': 'BRONX5',
        'SA Bronx 5 Lower': 'BRONX5',
        'SA Bushwick': 'BUSHWICK',
        'SA Cobble Hill': 'COBBLE_HILL',
        'SA Crown Heights': 'CROWN_HEIGHTS',
        'SA Ditmas Park Middle School': 'DITMAS_PARK_MIDDLE_SCHOOL',
        'SA East Flatbush Middle School': 'EAST_FLATBUSH_MIDDLE_SCHOOL',
        'SA Far Rockaway': 'FAR_ROCKAWAY',
        'SA Far Rockaway Middle School': 'FAR_ROCKAWAY_MIDDLE_SCHOOL',
        'SA Flatbush': 'FLATBUSH',
        'SA Hamilton Heights Middle School': 'HARLEM6',
        'SA Harlem 1': 'HARLEM1',
        'SA Harlem 2': 'HARLEM2',
        'SA Harlem 3': 'HARLEM3',
        'SA Harlem 4': 'HARLEM4',
        'SA Harlem 5': 'HARLEM5',
        'SA Harlem 6': 'HARLEM6',
        'SA Harlem East': 'HARLEM_EAST',
        'SA Harlem East Middle School': 'HARLEM_EAST',
        'SA Harlem North Central': 'HARLEM_NORTH_CENTRAL',
        'SA Harlem North Central Middle School': 'HARLEM_NORTH_CENTRAL',
        'SA Harlem West': 'HARLEM_WEST',
        'SA Harlem West Middle School': 'HARLEM_WEST',
        'SA Harlem North West': 'HARLEM_NORTH_WEST',
        'SA Harlem North West Middle School': 'HARLEM_NORTH_WEST',
        'SA Hells Kitchen': 'HELLS_KITCHEN',
        'SA Hell\'s Kitchen': 'HELLS_KITCHEN',
        'SA High School of the Liberal Arts - Manhattan': 'HIGH_SCHOOL_OF_THE_LIBERAL_ARTS_-_MANHATTAN',
        'SA High School of the Liberal Arts-Manhattan': 'HIGH_SCHOOL_OF_THE_LIBERAL_ARTS_-_MANHATTAN',
        'SA High School of the Liberal Arts - Harlem': 'HIGH_SCHOOL_OF_THE_LIBERAL_ARTS_-_HARLEM',
        'SA High School of the Liberal Arts-Harlem': 'HIGH_SCHOOL_OF_THE_LIBERAL_ARTS_-_HARLEM',
        'SA High School of the Liberal Arts - Brooklyn': 'HIGH_SCHOOL_OF_THE_LIBERAL_ARTS_-_BROOKLYN',
        'SA High School of the Liberal Arts-Brooklyn': 'HIGH_SCHOOL_OF_THE_LIBERAL_ARTS_-_BROOKLYN',
        'SA Hudson Yards': 'HUDSON_YARDS',
        'SA Hudson Yards Middle School': 'HUDSON_YARDS_MIDDLE_SCHOOL',
        'SA Kingsbridge Heights': 'KINGSBRIDGE_HEIGHTS',
        'SA Lafayette Middle School': 'LAFAYETTE_MIDDLE_SCHOOL',
        'SA Midtown West Middle School': 'MIDTOWN_WEST',
        'SA Myrtle Middle School': 'MYRTLE_MIDDLE_SCHOOL',
        'SA Norwood': 'NORWOOD',
        'SA Ozone Park Middle School': 'OZONE_PARK_MIDDLE_SCHOOL',
        'SA Prospect Heights': 'PROSPECT_HEIGHTS',
        'SA Queens Village': 'QUEENS_VILLAGE',
        'SA Rosedale': 'ROSEDALE',
        'SA Rockaway Park Middle School': 'ROCKAWAY_PARK_MIDDLE_SCHOOL',
        'SA South Jamaica': 'SOUTH_JAMAICA',
        'SA Sheepshead Bay': 'SHEEPSHEAD_BAY',
        'SA Springfield Gardens Middle School': 'SPRINGFIELD_GARDENS',
        'SA Springfield Gardens MS': 'SPRINGFIELD_GARDENS',
        'SA Springfield Gardens': 'SPRINGFIELD_GARDENS',
        'SA Union Square': 'UNION_SQUARE',
        'SA Upper West': 'UPPER_WEST',
        'SA Washington Heights': 'WASHINGTON_HEIGHTS',
        'SA Williamsburg': 'WILLIAMSBURG',
        }
    return schools

def _replace_with_keys(df, column, dictionary):
    new_df = pd.DataFrame()
    for key, value in dictionary.items():
        temp_df = df[df[column] == value].copy()
        temp_df[column] = key
        new_df = pd.concat([new_df, temp_df])
    return new_df

def _get_table_data(basetable=''):
    raw_basetable = pd.read_csv(basetable)
    df = raw_basetable
    df = df[~df['grade'].isin(['PK', 'Not In School'])]
    df = df[~df['grade'].isnull()]
    df['scholar_grade'] = np.where(df['grade']=='K','0',df['grade']).astype(int)
    df['latitude'] = df['students_home__latitude__s']
    df['longitude'] = df['students_home__longitude__s']
    df['school'] = df['accepted_school']
    df = _get_commute_time(_retrieve_nearest_census_tract_numbers(df,local_mode=True),local_mode=True)
    df['intercept'] = 1
    df['es_school'] = df['scholar_grade'].isin(np.arange(0,5)).astype(int)
    df['ms_school'] = df['scholar_grade'].isin(np.arange(5,6)).astype(int)
    df['log_commute'] = np.log(df['commute_time'])
    df['log_commute_square'] = df['log_commute'] ** 2
    df['log_commute_third'] = df['log_commute'] ** 3
    df = df[['intercept','yield','uniform_ordered','accepted_first_rank','had_enrolled_sib','ell_status','homeless_status','es_school','ms_school','orientation_rsvp','virtual_event_attended','in_person_event_attended',
            'scholar_grade','commute_time',
            'log_commute','log_commute_square','log_commute_third',
            'school','utm_source_bucketing']]
    df = df.dropna()
    return df

def set_predictors(current_predictors):
    predictors = current_predictors
    return predictors

def set_target(current_target):
    target = current_target
    return target

def setup_training_data(df, predictors, target):
    X = df[predictors]
    Y = df[target]
    return X, Y

def test_train_split(X, Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=0)
    return X_train, X_test, y_train, y_test

In [3]:
df = _get_table_data('train_basetable.csv')
df

,intercept,yield,uniform_ordered,accepted_first_rank,had_enrolled_sib,ell_status,homeless_status,es_school,ms_school,orientation_rsvp,virtual_event_attended,in_person_event_attended,scholar_grade,commute_time,log_commute,log_commute_square,log_commute_third,school,utm_source_bucketing
0,1,0,0,1,0,1,0,1,0,0,0,0,1,46.616667,3.841958,14.760642,56.709770,SA Bronx 4,Organic / No Tracking
1,1,0,0,1,0,1,0,1,0,0,0,0,1,46.616667,3.841958,14.760642,56.709770,SA Bronx 4,Organic / No Tracking
2,1,0,0,0,0,1,0,1,0,0,0,0,3,80.066667,4.382860,19.209458,84.192360,SA Washington Heights,Google Branded Search
3,1,1,1,1,0,1,0,1,0,1,0,0,3,5.100000,1.629241,2.654425,4.324696,SA Far Rockaway,Organic / No Tracking
4,1,0,0,0,0,0,0,1,0,0,0,0,2,34.000000,3.526361,12.435219,43.851064,SA Bronx 2,Organic / No Tracking
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19269,1,0,0,1,0,0,0,1,0,1,0,0,2,8.800000,2.174752,4.729545,10.285586,SA Washington Heights,Organic / No Tracking
19270,1,0,0,1,0,1,0,1,0,0,0,0,2,27.166667,3.301991,10.903143,36.002076,SA Bergen Beach,Organic / No Tracking
19271,1,0,0,1,0,1,0,1,0,0,0,0,0,12.283333,2.508243,6.291285,15.780073,SA Bronx 1,Google Branded Search
19272,1,0,1,1,0,0,0,1,0,0,0,0,0,28.200000,3.339322,11.151071,37.237017,SA Harlem 3,Organic / No Tracking


In [4]:
target = set_target(['yield'])
predictors = set_predictors(['uniform_ordered','accepted_first_rank','had_enrolled_sib',
                            'orientation_rsvp','virtual_event_attended','in_person_event_attended'])
X, y =  setup_training_data(df, predictors, target)
X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.float32)

In [8]:
# Regression model
linear_reg_model = PyroModule[nn.Linear](6, 1)

# Define loss and optimize
loss_fn = torch.nn.MSELoss(reduction='sum')
optim = torch.optim.Adam(linear_reg_model.parameters(), lr=0.05)
num_iterations = 1500


def train(x_data,y_data):
    # run the model forward on the data
    y_pred = linear_reg_model(x_data).squeeze(-1)
    # calculate the mse loss
    loss = loss_fn(y_pred, y_data)
    # initialize gradients to zero
    optim.zero_grad()
    # backpropagate
    loss.backward()
    # take a gradient step
    optim.step()
    return loss

for j in range(num_iterations):
    loss = train(X,y)
    if (j + 1) % 50 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss.item()))

# Inspect learned parameters
print("Learned parameters:")
for name, param in linear_reg_model.named_parameters():
    print(name, param.data.numpy())

[iteration 0050] loss: 71834256.0000
[iteration 0100] loss: 71754288.0000
[iteration 0150] loss: 71752720.0000
[iteration 0200] loss: 71752696.0000
[iteration 0250] loss: 71752696.0000
[iteration 0300] loss: 71752680.0000
[iteration 0350] loss: 71752680.0000
[iteration 0400] loss: 71752680.0000
[iteration 0450] loss: 71752680.0000
[iteration 0500] loss: 71752680.0000
[iteration 0550] loss: 71752680.0000
[iteration 0600] loss: 71752680.0000
[iteration 0650] loss: 71752680.0000
[iteration 0700] loss: 71752680.0000
[iteration 0750] loss: 71752680.0000
[iteration 0800] loss: 71752680.0000
[iteration 0850] loss: 71752680.0000
[iteration 0900] loss: 71752680.0000
[iteration 0950] loss: 71752680.0000
[iteration 1000] loss: 71752680.0000
[iteration 1050] loss: 71752680.0000
[iteration 1100] loss: 71752680.0000
[iteration 1150] loss: 71752680.0000
[iteration 1200] loss: 71752680.0000
[iteration 1250] loss: 71752680.0000
[iteration 1300] loss: 71752680.0000
[iteration 1350] loss: 71752680.0000
[

In [10]:
fit = df.copy()
fit["mean"] = linear_reg_model(X).detach().cpu().numpy()

In [12]:
from pyro.nn import PyroSample


class BayesianRegression(PyroModule):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = PyroModule[nn.Linear](in_features, out_features)
        self.linear.weight = PyroSample(dist.Normal(0., 1.).expand([out_features, in_features]).to_event(2))
        self.linear.bias = PyroSample(dist.Normal(0., 10.).expand([out_features]).to_event(1))

    def forward(self, x, y=None):
        sigma = pyro.sample("sigma", dist.Uniform(0., 10.))
        mean = self.linear(x).squeeze(-1)
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mean, sigma), obs=y)
        return mean

In [16]:
from pyro.infer.autoguide import AutoDiagonalNormal

model = BayesianRegression(6, 1)
guide = AutoDiagonalNormal(model)

In [19]:
from pyro.infer import SVI, Trace_ELBO


adam = pyro.optim.Adam({"lr": 0.03})
svi = SVI(model, guide, adam, loss=Trace_ELBO())

In [24]:
pyro.clear_param_store()
for j in range(num_iterations):
    # calculate the loss and take a gradient step
    loss = svi.step(X, y)
    if j % 100 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(df)))

ValueError: at site "obs", invalid log_prob shape
  Expected [19273], actual [19273, 19273]
  Try one of the following fixes:
  - enclose the batched tensor in a with pyro.plate(...): context
  - .to_event(...) the distribution being sampled
  - .permute() data dimensions